<div align="center">

# **DecisionTree Project Notebook**

</div>

### **Notebook Purpose**
This notebook shows how to build a Decision Tree from scratch. You will learn how to create, train and test a tree, and understand things like entropy, Gini, how the tree creates and works, and how to make it simpler using pruning.

### **Learning Goals**

* Build the Decision Tree and Node classes with full features
* Learn and code how to split data using Gini and Entropy
* Understand how the tree makes predictions and moves through nodes
* Use pruning to make the model simpler and better
* Check your code with clear and complete unit tests

### **Tasks**
Finish all the TODO parts in this notebook to create a Decision Tree classifier and pass all the tests.

# **Libraries Imports**
Import required libraries and add the DecisionTree and Node classes.

In [1]:
# ============================================================================
# LIBRARY IMPORTS
# ============================================================================

import numpy as np
import pandas as pd
import unittest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Set random seed for reproducibility
np.random.seed(42)

# ============================================================================
# DECISION TREE CLASSES IMPORT
# ============================================================================

from DT_Library import Node
from DT_Library import DecisionTree

## **Part 1 — Node Class**

### **What is a Node?**

A Node is one part of the decision tree. It stores the feature to split on, its children, and the final value prediction.  
(It can also be a leaf node.)

The Node class with TODOs is in [DT_Library](DT_Library.py) — complete it and tests will check if it works correctly.


In [2]:
# ============================================================================
# TESTS FOR NODE CLASS
# ============================================================================

class TestNodeInit(unittest.TestCase):

    def test_default_initialization(self):
        # Test default constructor behavior
        node = Node()
        self.assertTrue(hasattr(node, 'feature'))
        self.assertTrue(hasattr(node, 'children'))

    def test_feature_assignment(self):
        # Test feature parameter assignment
        node = Node(feature="age")
        self.assertEqual(node.feature, "age")
        self.assertIsNone(node.children)

    def test_children_assignment(self):
        # Test children parameter assignment
        child_list = [Node(), Node()]
        node = Node(children=child_list)
        self.assertEqual(len(node.children), 2)
        self.assertIsInstance(node.children[0], Node)

# Run the tests
runner = unittest.TextTestRunner(verbosity=2)
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestNodeInit)
result = runner.run(suite)

print("\nAll tests passed." if result.wasSuccessful() else "\nSome tests failed.")

test_children_assignment (__main__.TestNodeInit.test_children_assignment) ... ok
test_default_initialization (__main__.TestNodeInit.test_default_initialization) ... ok
test_feature_assignment (__main__.TestNodeInit.test_feature_assignment) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.018s

OK



All tests passed.


## **Part 2 — DecisionTree Hyperparameters**

### **Hyperparameters**

Hyperparameters control how the tree learns from data.

* **max_Depth** – maximum levels the tree can grow. A shallow tree may **underfit**, a very deep tree may **overfit**.
* **min_Samples** – minimum samples required to split a node.
* **pruning_threshold** – limits how small improvements must be to keep splitting. Helps reduce overfitting.
* **mode** – choice of splitting method (Gini or Entropy). Different methods can affect tree decisions.

You can add other hyperparameters if needed to improve your class.

A DecisionTree class with TODOs is in [DT_Library](DT_Library.py) — complete it and tests will check if it handles hyperparameters correctly.


In [3]:
# ============================================================================
# TESTS FOR DECISIONTREE CONSTRUCTOR
# ============================================================================

class TestDecisionTreeInit(unittest.TestCase):

    def test_constructor_attributes_exist(self):
        # Check required attributes are created
        dt = DecisionTree()
        self.assertTrue(hasattr(dt, 'mode'))
        self.assertTrue(hasattr(dt, 'max_Depth'))
        self.assertTrue(hasattr(dt, 'min_Samples'))
        self.assertTrue(hasattr(dt, 'pruning_threshold'))
        self.assertTrue(hasattr(dt, 'root'))

    def test_default_values_assignment(self):
        # Verify default parameter values are stored correctly
        dt = DecisionTree()
        self.assertEqual(dt.max_Depth, float("inf"))
        self.assertIsNone(dt.root)

    def test_custom_parameters_override_defaults(self):
        # Custom values should replace defaults
        dt = DecisionTree(mode="gini", max_Depth=10, min_Samples=5, pruning_threshold=0.01)
        self.assertEqual(dt.mode, "gini")
        self.assertEqual(dt.max_Depth, 10)
        self.assertEqual(dt.min_Samples, 5)
        self.assertEqual(dt.pruning_threshold, 0.01)
        self.assertIsNone(dt.root)

# Run the tests
runner = unittest.TextTestRunner(verbosity=2)
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestDecisionTreeInit)
result = runner.run(suite)

print("\nAll tests passed." if result.wasSuccessful() else "\nSome tests failed.")

test_constructor_attributes_exist (__main__.TestDecisionTreeInit.test_constructor_attributes_exist) ... ok
test_custom_parameters_override_defaults (__main__.TestDecisionTreeInit.test_custom_parameters_override_defaults) ... ok
test_default_values_assignment (__main__.TestDecisionTreeInit.test_default_values_assignment) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.011s

OK



All tests passed.


## **Part 3 — Splitting Criteria**

### **How the Tree Chooses Splits**

This part shows how the tree decides the best feature to split. We use measures like **Gini** and **Information Gain** to guide these decisions.

## **Part 3.1 — Entropy & Information Gain**

### **What is Entropy and Information Gain?**

* **Entropy** measures how mixed the classes are. For a target variable (Y) with classes (c):

$$Entropy(Y) = -\sum_{c} p(c) \log p(c)$$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **NOTE:** p(c) is the probability of class (c).

* **Conditional Entropy** shows the remaining uncertainty after splitting by a feature (X) with values (v):

$$Entropy(Y|X) = \sum_{v} P(X=v) \cdot Entropy(Y|X=v)$$

* **Information Gain** tells how much uncertainty is reduced by splitting on a feature:

$$GAIN(Y, X) = Entropy(Y) - Entropy(Y|X)$$

* Entropy is high when classes are mixed equally, low when one class dominates
* Information Gain shows how useful a feature is for reducing uncertainty
* *GAIN(Y, X) = 0* -> feature gives no information
* *GAIN(Y, X) = Entropy(Y)* -> feature perfectly separates classes

*Entropy* and *Information Gain* functions with TODOs are in [DT_Library](DT_Library.py) — complete them so that the tests check your functions return correctly.


In [4]:
# ============================================================================
# TESTS FOR INFORMATION GAIN
# ============================================================================

class TestInformationGain(unittest.TestCase):

    def test_returns_numeric_value(self):
        Y_simple = pd.Series([0, 1, 0, 1, 1, 0], name='target')
        X_simple = pd.DataFrame({
            'feature_simple': [1, 5, 3, 5, 3, 1],
            'other': [1, 2, 4, 4, 5, 1]
        })
        feature_simple = 'feature_simple'

        dt = DecisionTree(mode="gain")

        result = dt._information_Gain(feature_simple, X_simple, Y_simple)
        if isinstance(result, tuple):
            result = result[0]
        self.assertIsNotNone(result, "Method should return a value, not None")
        self.assertIsInstance(result, (int, float, np.number), "Result must be numeric")
        self.assertTrue(np.isfinite(result), "Result must be finite")

    def test_constant_feature_zero_gain(self):
        Y_mixed = pd.Series([0, 1, 0, 1, 1, 0], name='target')
        X_constant = pd.DataFrame({
            'constant_feature': [5, 5, 5, 5, 5, 5],
            'other': [1, 2, 3, 4, 5, 6]
        })
        feature_constant = 'constant_feature'

        dt = DecisionTree(mode="gain")

        result = dt._information_Gain(feature_constant, X_constant, Y_mixed)
        if isinstance(result, tuple):
            result = result[0]
        self.assertIsInstance(result, (int, float, np.number))
        self.assertAlmostEqual(result, 0.0, places=10, msg="Constant feature should give zero information gain")

    def test_pure_classes_zero_or_low_gain(self):
        # When Y is pure (all same class), gain should be low
        Y_pure = pd.Series([1, 1, 1, 1], name='target')
        X_any = pd.DataFrame({
            'any_feature': [0, 0, 1, 1],
            'other': [10, 20, 30, 40]
        })
        feature_any = 'any_feature'

        dt = DecisionTree(mode="gain")

        result = dt._information_Gain(feature_any, X_any, Y_pure)
        if isinstance(result, tuple):
            result = result[0]
        self.assertIsInstance(result, (int, float, np.number))
        self.assertLessEqual(result, 1e-10, "Pure classes should give very low information gain")

    def test_exact_information_gain_calculation(self):
        # Test with exact computable values
        Y_test = pd.Series([0, 0, 1, 1, 0, 0, 0], name='target')
        X_test = pd.DataFrame({
            'perfect_split': [0, 0, 1, 0, 2, 2, 2],
            'other': [10, 20, 30, 40, 10, 5, 15]
        })
        feature_test = 'perfect_split'

        dt = DecisionTree(mode="gain")

        result = dt._information_Gain(feature_test, X_test, Y_test)
        if isinstance(result, tuple):
            result = result[0]
        self.assertIsInstance(result, (int, float, np.number))
        self.assertAlmostEqual(result, 0.46956521111470667, places=3, msg=f"That {result} is not valid!")

# Run the tests
if __name__ == "__main__":
    runner = unittest.TextTestRunner(verbosity=2)
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestInformationGain)
    result = runner.run(suite)

    print("\nAll tests passed." if result.wasSuccessful() else "\nSome tests failed.")

test_constant_feature_zero_gain (__main__.TestInformationGain.test_constant_feature_zero_gain) ... ok
test_exact_information_gain_calculation (__main__.TestInformationGain.test_exact_information_gain_calculation) ... ok
test_pure_classes_zero_or_low_gain (__main__.TestInformationGain.test_pure_classes_zero_or_low_gain) ... ok
test_returns_numeric_value (__main__.TestInformationGain.test_returns_numeric_value) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.033s

OK



All tests passed.


## **Part 3.2 — Gini Impurity & Gini Split**

### **What is Gini Impurity and Gini Split?**

* **Gini Impurity** measures how impure or mixed the classes are. For a target variable (Y) with classes (c):

$$Gini(Y) = 1 - \sum_{c} p(c)^2$$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **NOTE:** p(c) is the probability of class (c).

* **Gini Split** calculates the weighted impurity after splitting by a feature (X) with values (v):

$$Gini\_Split(Y, X) = \sum_{v} \frac{|Y_v|}{|Y|} \cdot Gini(Y_v)$$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **NOTE:** |Y_v| is the number of samples where X=v, and Y_v are the corresponding labels.

* Gini ranges from 0 (pure) to 0.5 (maximum impurity for binary classes)
* Lower Gini Split values indicate better splits

*Gini Impurity* and *Gini Split* functions with TODOs are in [DT_Library](DT_Library.py) — complete them so that the tests check your functions return correctly.

In [6]:
# ============================================================================
# TESTS FOR GINI SPLIT
# ============================================================================

class TestGiniSplit(unittest.TestCase):

    def test_returns_numeric_value(self):
        Y_simple = np.array([0, 1, 0, 1, 1, 0])
        X_simple = pd.DataFrame({
            'feature_simple': [1, 5, 3, 5, 3, 1],
            'other': [1, 2, 4, 4, 5, 1]
        })
        feature_simple = 'feature_simple'

        dt = DecisionTree()

        result = dt._gini_Split(feature_simple, X_simple, Y_simple)
        if isinstance(result, tuple):
            result = result[0]
        self.assertIsNotNone(result, "Method should return a value, not None")
        self.assertIsInstance(result, (int, float, np.number), "Result must be numeric")
        self.assertTrue(np.isfinite(result), "Result must be finite")

    def test_constant_feature_original_gini(self):
        Y_mixed = np.array([0, 1, 0, 1, 1, 0])
        X_constant = pd.DataFrame({
            'constant_feature': [5, 5, 5, 5, 5, 5],
            'other': [1, 2, 3, 4, 5, 6]
        })
        feature_constant = 'constant_feature'

        dt = DecisionTree()

        result = dt._gini_Split(feature_constant, X_constant, Y_mixed)
        if isinstance(result, tuple):
            result = result[0]
        self.assertIsInstance(result, (int, float, np.number))
        self.assertAlmostEqual(result, 0.5, places=10, msg="Constant feature should return original gini impurity")

    def test_pure_classes_zero_gini(self):
        Y_pure = np.array([1, 1, 1, 1])
        X_any = pd.DataFrame({
            'any_feature': [0, 0, 1, 1],
            'other': [10, 20, 30, 40]
        })
        feature_any = 'any_feature'

        dt = DecisionTree()

        result = dt._gini_Split(feature_any, X_any, Y_pure)
        if isinstance(result, tuple):
            result = result[0]
        self.assertIsInstance(result, (int, float, np.number))
        self.assertLessEqual(result, 1e-10, "Pure classes should give zero gini impurity")

    def test_exact_gini_split_calculation(self):
        Y_test = np.array([0, 0, 1, 1, 0, 0, 0])
        X_test = pd.DataFrame({
            'split_feature': [0, 0, 1, 0, 2, 2, 2],
            'other': [10, 20, 30, 40, 10, 5, 15]
        })
        feature_test = 'split_feature'

        dt = DecisionTree()

        result = dt._gini_Split(feature_test, X_test, Y_test)
        if isinstance(result, tuple):
            result = result[0]
        self.assertIsInstance(result, (int, float, np.number))
        self.assertAlmostEqual(result, 0.19047619047619047, places=3, msg=f"That {result} is not valid!")

# Run the tests
if __name__ == "__main__":
    runner = unittest.TextTestRunner(verbosity=2)
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestGiniSplit)
    result = runner.run(suite)

    print("\nAll tests passed." if result.wasSuccessful() else "\nSome tests failed.")

test_constant_feature_original_gini (__main__.TestGiniSplit.test_constant_feature_original_gini) ... ok
test_exact_gini_split_calculation (__main__.TestGiniSplit.test_exact_gini_split_calculation) ... ok
test_pure_classes_zero_gini (__main__.TestGiniSplit.test_pure_classes_zero_gini) ... ok
test_returns_numeric_value (__main__.TestGiniSplit.test_returns_numeric_value) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.026s

OK



All tests passed.


## **Part 3.3 — Best Feature Selection**

### **What is Best Feature Selection?**

This process finds the feature that gives the best split according to the chosen criterion (Gini or Information Gain).  
Calculate splitting criterion for each feature in the dataset and then compare all features to select the feature with the best score.

- For **Information Gain** higher values are better
- For **Gini Split** lower values are better
- The function should return a Node object containing the best feature information

*Best Feature Selection* function with TODOs is in [DT_Library](DT_Library.py) — complete it so that the tests check your function works correctly with both Gini and Information Gain modes.

In [7]:
# ============================================================================
# TESTS FOR BEST FEATURE SELECTION
# ============================================================================

class TestBestFeature(unittest.TestCase):

    def test_returns_node_object(self):
        # Must return a Node object not None
        Y_simple = np.array([0, 1, 0, 1, 1, 0])
        X_simple = pd.DataFrame({
            'feature1': [1, 5, 3, 5, 3, 1],
            'feature2': [2, 3, 1, 3, 1, 2]
        })

        dt = DecisionTree(mode="gain")
        result = dt._get_best_Feature(X_simple, Y_simple)

        self.assertIsNotNone(result, "Method should return a Node, not None")
        self.assertIsInstance(result, Node, "Result must be a Node object")

    def test_gain_mode_selects_best_feature(self):
        # Gain mode should select feature with highest information gain
        Y_test = np.array([0, 0, 1, 1, 0, 0])
        X_test = pd.DataFrame({
            'good_feature': [0, 0, 1, 1, 0, 0],  # perfect correlation
            'bad_feature': [1, 1, 1, 1, 1, 1]   # constant feature
        })

        dt = DecisionTree(mode="gain")
        result = dt._get_best_Feature(X_test, Y_test)

        self.assertIsInstance(result, Node)
        # Should select the good feature that provides information
        self.assertIsNotNone(result.feature, "Selected feature should not be None")

    def test_gini_mode_selects_best_feature(self):
        # Gini mode should select feature with lowest gini split
        Y_test = np.array([0, 0, 1, 1, 0, 0])
        X_test = pd.DataFrame({
            'good_feature': [0, 0, 1, 1, 0, 0],  # good separation
            'bad_feature': [2, 2, 2, 2, 2, 2]   # constant feature
        })

        dt = DecisionTree(mode="gini")
        result = dt._get_best_Feature(X_test, Y_test)

        self.assertIsInstance(result, Node)
        # Should select the good feature that reduces impurity
        self.assertIsNotNone(result.feature, "Selected feature should not be None")

    def test_different_modes_same_data(self):
        # Both modes should work on same dataset
        Y_test = np.array([0, 1, 0, 1, 1, 0, 0])
        X_test = pd.DataFrame({
            'feature_a': [1, 2, 1, 2, 1, 2, 1],
            'feature_b': [5, 5, 3, 3, 5, 3, 5]
        })

        dt_gain = DecisionTree(mode="gain")
        dt_gini = DecisionTree(mode="gini")

        result_gain = dt_gain._get_best_Feature(X_test, Y_test)
        result_gini = dt_gini._get_best_Feature(X_test, Y_test)

        self.assertIsInstance(result_gain, Node)
        self.assertIsInstance(result_gini, Node)
        # Both should return valid Node objects
        self.assertTrue(hasattr(result_gain, 'feature'))
        self.assertTrue(hasattr(result_gini, 'feature'))

# Run the tests
runner = unittest.TextTestRunner(verbosity=2)
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestBestFeature)
result = runner.run(suite)

print("\nAll tests passed." if result.wasSuccessful() else "\nSome tests failed.")

test_different_modes_same_data (__main__.TestBestFeature.test_different_modes_same_data) ... ok
test_gain_mode_selects_best_feature (__main__.TestBestFeature.test_gain_mode_selects_best_feature) ... ok
test_gini_mode_selects_best_feature (__main__.TestBestFeature.test_gini_mode_selects_best_feature) ... ok
test_returns_node_object (__main__.TestBestFeature.test_returns_node_object) ... 

ok

----------------------------------------------------------------------
Ran 4 tests in 0.042s

OK



All tests passed.


## **Part 4 — Tree Building & Training**

### **How the Tree Learns**

You will build the full decision tree by splitting data step by step until stopping rules are met.

- **fit()** initializes the training process and sets up the root node.
- **_create_Tree()** recursively builds tree structure by finding best splits.
- The process stops when the tree reaches its maximum depth, when there are too few samples to split, or when all samples in a node belong to one class.
- Each node saves the best feature and links to its children for different feature values.

*fit* and *_create_Tree* functions with TODOs are in [DT_Library](DT_Library.py) — complete them so that the tests check your functions build valid tree structures correctly.

In [8]:
# ============================================================================
# TESTS FOR TREE BUILDING & TRAINING
# ============================================================================

class TestTreeBuilding(unittest.TestCase):

    # ===============================
    # HELPER FUNCTIONS
    # ===============================

    def calc_depth(self, node):
        if node is None or not hasattr(node, "children") or not node.children:
            return 0
        return 1 + max(self.calc_depth(child) for child in node.children)

    def count_nodes(self, node):
        if node is None:
            return 0
        if not hasattr(node, "children") or not node.children:
            return 1
        return 1 + sum(self.count_nodes(child) for child in node.children)

    # ===============================
    # TEST FUNCTIONS
    # ===============================

    def test_fit_creates_root_node(self):
        # Check if fit() creates a valid root node
        Y_simple = np.array([0, 0, 0, 0, 1, 1, 1, 2])
        X_simple = pd.DataFrame({
            'sequential': [1, 2, 3, 4, 5, 6, 7, 8],  # simple increasing values
            'noise': [10, 11, 12, 13, 14, 15, 16, 17]
        })

        dt = DecisionTree(mode="gain")
        dt.fit(X_simple, Y_simple)

        self.assertIsNotNone(dt.root, "Root node should exist after training")
        self.assertIsInstance(dt.root, Node)
        self.assertIsNotNone(dt.root.feature, "Root should choose a feature to split on")
        self.assertIn(dt.root.feature, X_simple.columns, "Root feature must be one of the dataset columns")

    def test_perfectly_balanced_tree(self):
        # Check if a balanced dataset creates a balanced binary tree
        Y_balanced = np.array([0, 0, 1, 1, 0, 0, 1, 1])
        X_balanced = pd.DataFrame({
            'binary_split': [0, 0, 1, 1, 0, 0, 1, 1],
            'secondary': [0, 1, 0, 1, 1, 0, 1, 0]
        })

        dt_gain = DecisionTree(mode="gain", max_Depth=3)
        dt_gini = DecisionTree(mode="gini", max_Depth=3)

        dt_gain.fit(X_balanced, Y_balanced)
        dt_gini.fit(X_balanced, Y_balanced)

        self.assertEqual(dt_gain.root.feature, 'binary_split', "Should pick the best splitting feature")
        self.assertEqual(dt_gini.root.feature, 'binary_split', "Should pick the best splitting feature")

    def test_early_stopping_min_samples(self):
        # Check that min_Samples stops further splitting
        Y = np.array([0, 0, 1, 1, 2, 2, 1, 1])
        X = pd.DataFrame({
            'feature1': [0, 0, 1, 1, 0, 0, 1, 1],
            'feature2': [0, 0, 0, 1, 1, 1, 1, 0]
        })

        dt_strict = DecisionTree(mode="gain", min_Samples=5)
        dt_lenient = DecisionTree(mode="gain", min_Samples=2)

        dt_strict.fit(X, Y)
        dt_lenient.fit(X, Y)

        depth_strict = self.calc_depth(dt_strict.root)
        depth_lenient = self.calc_depth(dt_lenient.root)

        self.assertEqual(depth_strict, 1, "Tree with high min_Samples should grow less")
        self.assertEqual(depth_lenient, 2, "Tree with small min_Samples should grow deeper")

    def test_depth_limited_vs_unlimited(self):
        # Check depth limit works correctly
        Y = np.array([0, 1, 2, 1, 1, 3, 1, 1, 3, 0, 1, 2])
        X = pd.DataFrame({
            "feature1": [1, 2, 2, 2, 2, 2, 3, 3, 3, 1, 4, 4],
            "feature2": [0, 1, 2, 0, 1, 3, 0, 1, 3, 0, 1, 1],
            'feature3': [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1],
            'feature4': [0, 2, 0, 1, 1, 2, 1, 0, 0, 1, 2, 1]
        })

        dt_limited = DecisionTree(mode="gini", max_Depth=2)
        dt_unlimited = DecisionTree(mode="gini", max_Depth=float("inf"))

        dt_limited.fit(X, Y)
        dt_unlimited.fit(X, Y)

        depth_limited = self.calc_depth(dt_limited.root)
        depth_unlimited = self.calc_depth(dt_unlimited.root)

        self.assertEqual(depth_limited, 2, "Tree must not go deeper than max_Depth")
        self.assertGreaterEqual(depth_unlimited, depth_limited, "Unlimited tree should grow at least as deep")

    def test_node_count_comparison(self):
        # Compare number of nodes for different constraints
        Y = np.array([0, 1, 2, 1, 1, 3, 1, 1, 3, 0, 1, 2])
        X = pd.DataFrame({
            "feature1": [1, 2, 2, 2, 2, 2, 3, 3, 3, 1, 4, 4],
            "feature2": [0, 1, 2, 0, 1, 3, 0, 1, 3, 0, 1, 1],
            'feature3': [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1],
            'feature4': [0, 2, 0, 1, 1, 2, 1, 0, 0, 1, 2, 1]
        })

        dt_constrained = DecisionTree(mode="gain", max_Depth=2, min_Samples=5)
        dt_free = DecisionTree(mode="gain", max_Depth=3, min_Samples=2)

        dt_constrained.fit(X, Y)
        dt_free.fit(X, Y)

        n_constrained = self.count_nodes(dt_constrained.root)
        n_free = self.count_nodes(dt_free.root)

        self.assertGreaterEqual(n_free, n_constrained, f"Tree with fewer limits ({n_free}) should have at least as many nodes as constrained one ({n_constrained})")
        self.assertEqual(n_free, 13, "Free tree should have more nodes, including a full root and children")

    def test_single_feature_dominance(self):
        # Check that the tree picks the best possible feature
        Y_dominant = np.array([0, 0, 0, 1, 1, 1, 2, 2])
        X_dominant = pd.DataFrame({
            'perfect_feature': [1, 1, 1, 2, 2, 2, 3, 3],
            'random_feature': [9, 3, 7, 1, 5, 8, 2, 6],
            'constant_feature': [5, 5, 5, 5, 5, 5, 5, 5]
        })

        dt = DecisionTree(mode="gain", max_Depth=5)
        dt.fit(X_dominant, Y_dominant)

        self.assertEqual(dt.root.feature, 'perfect_feature', "Tree should pick the feature that perfectly matches the target")

# Run the tests
runner = unittest.TextTestRunner(verbosity=2)
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestTreeBuilding)
result = runner.run(suite)

print("\nAll tests passed." if result.wasSuccessful() else "\nSome tests failed.")

test_depth_limited_vs_unlimited (__main__.TestTreeBuilding.test_depth_limited_vs_unlimited) ... 

ok
test_early_stopping_min_samples (__main__.TestTreeBuilding.test_early_stopping_min_samples) ... ok
test_fit_creates_root_node (__main__.TestTreeBuilding.test_fit_creates_root_node) ... ok
test_node_count_comparison (__main__.TestTreeBuilding.test_node_count_comparison) ... ok
test_perfectly_balanced_tree (__main__.TestTreeBuilding.test_perfectly_balanced_tree) ... ok
test_single_feature_dominance (__main__.TestTreeBuilding.test_single_feature_dominance) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.259s

OK



All tests passed.


## **Part 5 — Tree Prediction & Navigation**

### **How the Tree Makes Predictions**

In this part, you'll implement how the trained decision tree predicts new samples. The tree uses its learned structure to follow paths from the **root** to the correct **leaf** node.

- **predict()** takes a dataset and returns a predicted label for each sample
- **_move_Tree()** moves through tree structure following the decision path
- The process begins at the root and continues until a leaf node is reached
- Each **leaf node** stores the final class label (or value) that becomes the prediction

*predict* and *_move_Tree* functions with TODOs are in [DT_Library](DT_Library.py) — complete them so that the tests check your functions navigate tree correctly and return accurate predictions.

In [9]:
# ============================================================================
# TESTS FOR TREE PREDICTION & NAVIGATION
# ============================================================================

class TestTreePrediction(unittest.TestCase):

    # ===============================
    # HELPER FUNCTIONS
    # ===============================

    def create_gini_tree(self):
        # Create a simple tree with clear class separation using Gini criterion
        Y_train = np.array([0, 0, 0, 2, 2, 1, 2, 2])
        X_train = pd.DataFrame({
            'feature1': [1, 1, 1, 2, 2, 2, 3, 3],
            'feature2': [9, 3, 7, 1, 5, 8, 2, 6],
            'feature3': [5, 5, 5, 5, 4, 5, 5, 5]
        })

        dt = DecisionTree(mode="gini", min_Samples=1, max_Depth=3)
        dt.fit(X_train, Y_train)
        return dt, X_train, Y_train

    def create_gain_tree(self):
        # Create a more complex tree using Information Gain criterion
        Y_train = np.array([0, 1, 2, 1, 1, 3, 1, 1, 3, 0, 1, 2])
        X_train = pd.DataFrame({
            "feature1": [1, 2, 2, 2, 2, 2, 3, 3, 3, 1, 4, 4],
            "feature2": [0, 1, 2, 0, 1, 3, 0, 1, 3, 0, 1, 1],
            'feature3': [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1],
            'feature4': [0, 2, 0, 1, 1, 2, 1, 0, 0, 1, 2, 1]
        })

        dt = DecisionTree(mode="gain", min_Samples=2, max_Depth=2)
        dt.fit(X_train, Y_train)
        return dt, X_train, Y_train

    # ===============================
    # TEST FUNCTIONS
    # ===============================

    def test_predict_output_shape(self):
        # Ensure predict() returns a valid array with the same length as input
        dt, X_train, Y_train = self.create_gini_tree()

        predictions = dt.predict(X_train)

        self.assertIsNotNone(predictions, "predict() should not return None")
        self.assertEqual(len(predictions), len(X_train), "predict() should return one output per input sample")

    def test_prediction_accuracy_on_training_data(self):
        # Check that the model achieves high accuracy on the training set
        dt, X_train, Y_train = self.create_gain_tree()

        predictions = dt.predict(X_train)

        accuracy = np.mean(predictions == Y_train)
        self.assertAlmostEqual(
            accuracy,
            0.91666666666,
            places=3,
            msg=f"Model should perform well on training data, got {accuracy:.2%}"
        )

    def test_predictions_use_correct_features_gain(self):
        # Verify that predictions are based on the most informative features (Information Gain)
        dt, X_train, Y_train = self.create_gain_tree()

        X_test = pd.DataFrame({
            'feature1': [1, 2, 2, 4],
            'feature2': [0, 0, 1, 3],
            'feature3': [0, 0, 0, 1],
            'feature4': [2, 2, 1, 0]
        })

        predictions = dt.predict(X_test)

        self.assertEqual(len(predictions), 4, "predict() should return a prediction for each test sample")

        for i, pred in enumerate(predictions):
            self.assertIn(pred, [0, 1, 3], f"Prediction {pred} should be one of the valid class labels")

        self.assertEqual(predictions[3], 3, "Sample 3 should be classified as class 3")

    def test_predictions_use_correct_features_gini(self):
        # Verify that predictions are based on key features (Gini criterion)
        dt, X_train, Y_train = self.create_gini_tree()

        X_test = pd.DataFrame({
            'feature1': [1, 3, 2],
            'feature2': [3, 2, 6],
            'feature3': [5, 4, 4]
        })

        predictions = dt.predict(X_test)

        self.assertEqual(len(predictions), 3, "predict() should return a prediction for each test sample")

        for i, pred in enumerate(predictions):
            self.assertIn(pred, [0, 2], f"Prediction {pred} should be one of the valid class labels")

        self.assertEqual(predictions[0], 0, "Sample 0 should be classified as class 0")

# Run the tests
runner = unittest.TextTestRunner(verbosity=2)
suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestTreePrediction)
result = runner.run(suite)

print("\nAll tests passed." if result.wasSuccessful() else "\nSome tests failed.")

test_predict_output_shape (__main__.TestTreePrediction.test_predict_output_shape) ... ok
test_prediction_accuracy_on_training_data (__main__.TestTreePrediction.test_prediction_accuracy_on_training_data) ... ok
test_predictions_use_correct_features_gain (__main__.TestTreePrediction.test_predictions_use_correct_features_gain) ... ok
test_predictions_use_correct_features_gini (__main__.TestTreePrediction.test_predictions_use_correct_features_gini) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.125s

OK



All tests passed.


## **Part 6 — Hyperparameter Optimization**

### **How to Optimize and Control the Tree**

In this part, you'll implement advanced techniques to find the best tree settings and prevent overfitting. These methods help create trees that work well on new, unseen data.

- **Grid search** tries many combinations of hyperparameters and picks the combination that gives the best validation performance. Proper tuning can reduce both underfitting and overfitting. you can use random search too.

**Note:** There are no tests for this part — you must implement and validate these functions yourself.

**Validation tips** (keep these results for your presentation):

* **Grid search:** Try many hyperparameter combinations, print the validation accuracies for each, compare them, and identify whether the chosen hyperparameters look truly optimal.
* **Keep your results:** Keep the printed outputs and a short written analysis — you will be asked about these in your presentation.

In [17]:
import os
import joblib
import pandas as pd
from itertools import product
from sklearn.model_selection import train_test_split
from DT_Library import DecisionTree

df = pd.read_csv("EuroRail_Survey_cleaned.csv")
X = df.drop("satisfaction", axis=1)
y = df["satisfaction"].map({"satisfied": 1, "dissatisfied": 0})

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=42)

param_grid = {
    "max_depth": [10, 12, 14],
    "mode": ["gain", "gini"],
    "min_Samples": [50, 60, 70],
    "pruning_threshold": [0.005, 0.01, 0.02],
}

ckpt_dir = "checkpoints"
os.makedirs(ckpt_dir, exist_ok=True)
best_model_path = os.path.join(ckpt_dir, "best_model.pkl")

best_accuracy = -1.0
best_params = None

for depth, mode, min_smp, threshold in product(
        param_grid["max_depth"],
        param_grid["mode"],
        param_grid["min_Samples"],
        param_grid["pruning_threshold"]):

    combo_name = f"d{depth}_m{mode}_min{min_smp}_th{str(threshold).replace('.', '_')}"
    combo_ckpt = os.path.join(ckpt_dir, f"{combo_name}.pkl")

    print("------------------------------------------------------")
    print(f"Combo: max_depth={depth}, mode={mode}, min_Samples={min_smp}, prune_th={threshold}")

    if os.path.exists(combo_ckpt):
        print("Found checkpoint. Loading existing model...")
        tree = joblib.load(combo_ckpt)
    else:
        print("No checkpoint. Training new model...")
        tree = DecisionTree(max_Depth=depth, mode=mode,
                            min_Samples=min_smp, pruning_threshold=threshold)
        tree.fit(X_train, y_train)
        joblib.dump(tree, combo_ckpt)
        print(f"Saved checkpoint for this combo at {combo_ckpt}")

    y_pred = tree.predict(X_val)
    accuracy = sum(yt == yp for yt, yp in zip(y_val, y_pred)) / len(y_val)
    print(f"Validation Accuracy = {accuracy:.4f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {
            "max_depth": depth,
            "mode": mode,
            "min_Samples": min_smp,
            "pruning_threshold": threshold
        }
        joblib.dump(tree, best_model_path)
        print(f"✅ New best model saved! Accuracy = {accuracy:.4f}")

print("======================================================")
print("Training finished.")
print(f"Best Validation Accuracy = {best_accuracy:.4f}")
print("Best Params:", best_params)

if os.path.exists(best_model_path):
    best_tree = joblib.load(best_model_path)
    y_test_pred = best_tree.predict(X_test)
    test_acc = sum(yt == yp for yt, yp in zip(y_test, y_test_pred)) / len(y_test)
    print(f"Test Accuracy of Best Model = {test_acc:.4f}")


------------------------------------------------------
Combo: max_depth=10, mode=gain, min_Samples=50, prune_th=0.005
No checkpoint. Training new model...
Saved checkpoint for this combo at checkpoints\d10_mgain_min50_th0_005.pkl
Validation Accuracy = 0.9141
✅ New best model saved! Accuracy = 0.9141
------------------------------------------------------
Combo: max_depth=10, mode=gain, min_Samples=50, prune_th=0.01
No checkpoint. Training new model...
Saved checkpoint for this combo at checkpoints\d10_mgain_min50_th0_01.pkl
Validation Accuracy = 0.9142
✅ New best model saved! Accuracy = 0.9142
------------------------------------------------------
Combo: max_depth=10, mode=gain, min_Samples=50, prune_th=0.02
No checkpoint. Training new model...
Saved checkpoint for this combo at checkpoints\d10_mgain_min50_th0_02.pkl
Validation Accuracy = 0.9109
------------------------------------------------------
Combo: max_depth=10, mode=gain, min_Samples=60, prune_th=0.005
No checkpoint. Training n

## **Part 7 — Data Cleaning & Preprocessing**

### **How to Prepare Real Data for Your Tree**

In this part, you'll **clean and preprocess the EuroRail_Survey dataset** to make it ready for decision tree modeling. Real-world data is often messy, incomplete, or inconsistent — proper preprocessing ensures reliable learning and accurate predictions.

* **Missing Values Handling:** Fill or remove incomplete records using smart strategies (e.g., median for numeric data or mode for categorical).
* **Duplicate Removal:** Detect and delete duplicate rows to avoid bias during training.
* **Outlier Detection:** Identify extreme or unrealistic values that could distort splits and model accuracy.
* **Feature Scaling:** Normalize or standardize numeric features (using *MinMaxScaler* or *StandardScaler*) to make feature comparisons fair.
* **Feature Selection:** Remove irrelevant or highly correlated attributes using correlation analysis or heatmaps to reduce redundancy.
* **Categorical Encoding:** Transform text-based features into numeric form with *LabelEncoder* or *OneHotEncoder*.
* **Binning:** Convert continuous variables into discrete bins to help the tree find clearer splitting patterns.

**Note:** Not all steps are mandatory — analyze your data and choose methods that improve accuracy and interpretability.

**Important:** Save your cleaned dataset and clearly document each preprocessing step you applied and why. Then, **compare model performance before and after preprocessing** to show how data preparation improved your decision tree.

In [10]:
# ============================================================================
# DATA CLEANING & PREPROCESSING
# ============================================================================

# Load the raw dataset
dataframe = pd.read_csv('EuroRail_Survey.csv')
print('\nDataset Info:')
print(dataframe.info())
print(dataframe.info(), '\n')

#this is to see how many duplicate rows we have in our dataframe:(you will see there are no duplications.)
print(dataframe[dataframe.duplicated()])


# list comprehension to see the numerical or categorical columns:
cat_col = [col for col in dataframe.columns if dataframe[col].dtype == 'object']
num_col = [col for col in dataframe.columns if dataframe[col].dtype != 'object']

print('Categorical columns:', cat_col)
print('Numerical columns:', num_col)

#to see the number of unique values for categorical features:
print(dataframe[cat_col].nunique())

print()
print()

null_percentages = round((dataframe.isnull().sum() / dataframe.shape[0]) * 100, 2)

print(null_percentages)

# ============================================================================
# DATA PREPROCESSING PIPELINE
# ============================================================================

# TODO: Apply data cleaning and preprocessing steps

# Step1:Deleting null containing rows:

#As i printed some info about the csv file,the missing datas in all the columns are less than 0.5%.
# the simplest way to clean this dirty ,is to delete those corresponding rows:

# this is not needed because we see that all the features have less than 0.5% for null values but i do this working for other csv files too:
np_array_2d = np.column_stack((null_percentages.index, null_percentages.to_numpy()))

# features array filtered by the percentage of null values less than 0.5%:
values = np_array_2d[:,1].astype(float)
filtered_null_percentage = np_array_2d[values < 0.5, 0]

dataframe = dataframe.dropna(subset=filtered_null_percentage)



# Step2: Calculating Outlier Boundaries and removing them:

for column in num_col:
    mean = dataframe[column].mean()
    std = dataframe[column].std()

    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std

    dataframe = dataframe[(dataframe[column] >= lower_bound) & (dataframe[column] <= upper_bound)]


# Step3: Deleting columns that don’t carry information for prediction
dataframe = dataframe.drop(columns=['id'])

# Save the cleaned dataset
dataframe.to_csv('EuroRail_Survey_cleaned.csv', index=False)

print('\nPreprocessing completed successfully!')
print('Cleaned dataset saved and ready for decision tree training.')


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129880 entries, 0 to 129879
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 129880 non-null  int64  
 1   Passenger Type                     129272 non-null  object 
 2   Age                                129244 non-null  float64
 3   Type of Trip                       129297 non-null  object 
 4   Ticket Class                       129263 non-null  object 
 5   Trip Distance                      129276 non-null  float64
 6   Seat comfort                       129295 non-null  float64
 7   Departure/Arrival time convenient  129270 non-null  float64
 8   Dining Service                     129329 non-null  float64
 9   Platform Location                  129261 non-null  float64
 10  Onboard wifi service               129241 non-null  float64
 11  Onboard entertainment   

## **Part 8 — Model Training & Testing**

### **Train Your Decision Tree on Clean Data**

Use your cleaned dataset to train and test your DecisionTree class. Calculate and display both **training and test accuracy**.

**Note:** If you achieve over 90% accuracy on test data, document your analysis explaining how you reached this performance level, and you'll receive bonus points.

In [ ]:
# ============================================================================
# MODEL TRAINING & TESTING
# ============================================================================

# these five lines were because i didnt want to again run the hyperparameter tuning section:(i had the result when i first run that part.)

# df = pd.read_csv("EuroRail_Survey_cleaned.csv")
# X = df.drop("satisfaction", axis=1)
# y = df["satisfaction"].map({"satisfied": 1, "dissatisfied": 0})

# X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=42)


# Combine the training and validation sets for final training
X_train_final = pd.concat([X_train, X_val])
y_train_final = pd.concat([y_train, y_val])



# Initialize the model with the best hyperparameters found from tuning
tree = DecisionTree(
    mode=best_params["mode"],
    max_Depth=best_params["max_depth"],
    min_Samples=best_params["min_Samples"],
    pruning_threshold=best_params["pruning_threshold"]
)

tree = DecisionTree(
    mode='gain',
    max_Depth=12,
    min_Samples=50,
    pruning_threshold=0.01
)

# Train the model on the combined training data (80%)
tree.fit(X_train_final, y_train_final)

# Evaluate the trained model on the training data itself (for train accuracy)
train_predict = tree.predict(X_train_final)

# Calculate the final train accuracy
train_accuracy = sum([1 if yt == yp else 0 for yt, yp in zip(y_train_final, train_predict)]) / len(y_train_final)

# Evaluate the trained model on the unseen test set (20%)
test_predict = tree.predict(X_test)

# Calculate the final test accuracy
test_accuracy = sum([1 if yt == yp else 0 for yt, yp in zip(y_test, test_predict)]) / len(y_test)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy (Unseen Data): {test_accuracy:.4f}")
print("\nModel training and testing completed!")


# TODO: Visualize your tree
export_decision_tree_to_pdf(tree, filename="euro_rail_tree_full")

export_decision_tree_to_pdf(tree, filename="euro_rail_tree_depth4", max_display_depth=4)

export_decision_tree_to_pdf(tree, filename="euro_rail_tree_collapsed", max_display_depth=6)

print('\nModel training and testing completed!')

Train Accuracy: 0.9274
Test Accuracy (Unseen Data): 0.9161

Model training and testing completed!


Rendered tree to: euro_rail_tree_full.pdf


Rendered tree to: euro_rail_tree_depth4.pdf


Rendered tree to: euro_rail_tree_collapsed.pdf

Model training and testing completed!


## **Part 9 — Post-Pruning in Decision Trees**

### **What is Post-Pruning?**

Post-pruning (also called "cost-complexity pruning") is a technique used to reduce overfitting in decision trees.  
After the tree is fully grown (possibly overfitting the training data), some branches are removed to make the tree simpler and improve its performance on unseen data.

**Why use it?**

- Fully grown trees can memorize noise in the training data.  
- Post-pruning simplifies the tree, making it more generalizable.  

**How to do it (briefly):**

1. Grow the full tree using your training data.  
2. Evaluate the performance of each subtree on a **validation set** (or using cross-validation).  
3. Remove branches/subtrees that do not improve performance on the validation data.  
4. Repeat until removing any branch would decrease validation accuracy.  

- **NOTE:** Post-pruning is applied **after the tree is fully built**, unlike pre-pruning which stops the tree growth early.

**Validation tips** (keep these results for your presentation):

* **Post-pruning:** Show the tree before and after pruning (a plot), report validation/test accuracy numbers, and explain if pruning improved generalization and why.
* **Keep your results:** Keep the printed outputs and a short written analysis — you will be asked about these in your presentation.

In [16]:
# Accuracy function to check the model's performance
def accuracy(x_val, y_val):
    y_pred = tree.predict(x_val)  # get predictions
    accuracy = (y_pred == y_val).sum() / len(y_val)  # calculate accuracy
    return accuracy

# Main post-pruning function
def postpruning(tree, x_val, y_val):
    initial_accuracy = accuracy(x_val, y_val)  # get initial accuracy
    # Nested function to prune nodes
    def node_prune(node):
        nonlocal initial_accuracy
        # Base case: if the node is a leaf, we can't prune further
        if node.is_leaf:
            return

        # Save the current children of the node (we may need to restore them)
        children_track = node.children

        # Temporarily make the node a leaf node by clearing its children
        node.children = []  
        node.is_leaf = True

        # Check the accuracy after pruning this node
        current_acc = accuracy(x_val, y_val)

        # If pruning decreased accuracy, revert back
        if current_acc < initial_accuracy:
            node.children = children_track  # restore the children
            node.is_leaf = False  # restore it to a non-leaf
            for child in node.children:
                node_prune(child)  # recurse to prune its children as well
        else:
            # If pruning improved or maintained accuracy, we keep it pruned
            initial_accuracy = current_acc

    # Start pruning from the root node
    node_prune(tree.root)

# Execute post-pruning
postpruning(tree, X_val, y_val)

# Want to see the accuracy on the train dataset after post-pruning:
ytrain_predict = tree.predict(X_train_final)
traindataset_accuracy =(ytrain_predict == y_train_final).sum()/len(y_train_final)

# Want to see the accuracy on the test dataset after post-pruning:
ytest_predict = tree.predict(X_test)
testdataset_accuracy = (ytest_predict == y_test).sum()/len(y_test)

print("The accuracy of the model after post-pruning for train dataset is:", traindataset_accuracy)
print("The accuracy of the model after post-pruning for test dataset is:", testdataset_accuracy)

# Want to see the tree after post-pruning:

# Exporting the tree after post-pruning as a full tree
export_decision_tree_to_pdf(tree, filename="postpruned_full_tree")

# Exporting the tree after post-pruning with a depth limit (depth 4)
export_decision_tree_to_pdf(tree, filename="postpruned_depth_4_tree", max_display_depth=4)

# Exporting the tree after post-pruning with a collapsed (pruned) version, showing up to depth 6
export_decision_tree_to_pdf(tree, filename="postpruned_collapsed_tree", max_display_depth=6)


The accuracy of the model after post-pruning for train dataset is: 0.9256742092818715
The accuracy of the model after post-pruning for test dataset is: 0.9167085484534199


Rendered tree to: postpruned_full_tree.pdf


Rendered tree to: postpruned_depth_4_tree.pdf


Rendered tree to: postpruned_collapsed_tree.pdf


'postpruned_collapsed_tree.pdf'

## **Part 10 — Decision Tree Visualization**

### **Why visualize a decision tree?**  
Visualizing a decision tree helps you understand how the tree splits the data, which features are important, and how decisions are made at each node.

**How to visualize a tree:**  

You can use several Python tools to draw your decision tree:

1. **Graphviz**  
   - Popular for creating clean, professional tree diagrams.
   - Can export as PDF, PNG, or SVG.

2. **NetworkX**  
   - Can build a tree as a graph and visualize nodes and edges.  
   - Allows more flexible customizations, colors, and layouts.  

3. **Other libraries**  
   - *matplotlib* with custom plotting  
   - *pydotplus* (works with Graphviz)  

**NOTE:**  
- Always try to include feature names and class labels in the visualization.  
- Use validation or pruning to avoid overly large trees that are hard to read.  
- For complex trees, exporting as PDF or using zoomable interactive plots is recommended.


In [11]:
# ============================================================================
# Tree Visualization
# ============================================================================
import graphviz
import html


#I have done this part using ChatGpt after coordiation with HeadTa:

def export_decision_tree_to_pdf(tree, filename="decision_tree", max_display_depth=None, collapse_threshold=None):
    """
    Export a DecisionTree (your custom class) to a PDF using graphviz with enhanced styling and layout.

    Parameters
    ----------
    tree : DecisionTree
        An instance of your DecisionTree (must have 'root' attribute of Node type).
    filename : str
        Base filename (without extension). Will produce filename + ".pdf".
    max_display_depth : int or None
        If set, only nodes up to this depth will be drawn. Deeper nodes replaced with a summary node.
    collapse_threshold : int or None
        If set, when a subtree has more than this many samples at its root (datasample_labels length),
        the subtree will be drawn but its children collapsed (useful for huge trees).
    """
    if tree.root is None:
        raise ValueError("Tree has no root. Fit the tree first.")

    dot = graphviz.Digraph(name="DecisionTree", format="pdf", graph_attr={"splines":"ortho", "rankdir":"LR"})  # Change to LR for horizontal layout
    dot.attr('node', shape='box', fontsize='10', fontname='Helvetica', width='0.3', height='0.3', style='filled', fillcolor='lightyellow')  # Added styling for better appearance

    _id_counter = {"c": 0}
    def new_id():
        _id_counter["c"] += 1
        return f"n{_id_counter['c']}"

    def safe_label(s):
        """Escape label text for Graphviz (HTML-like)."""
        # Convert any non-string value to string
        s = str(s)
        return html.escape(s).replace("\n", "<BR/>")

    def node_summary(node):
        """Return a short text summary for a node (feature / threshold / samples / class dist)."""
        # datasample_labels could be numpy array or list
        ds = getattr(node, "datasample_labels", None)
        n_samples = 0
        if ds is not None:
            try:
                n_samples = len(ds)
            except Exception:
                n_samples = 0
        # For leaf show answer
        if getattr(node, "is_leaf", False):
            ans = getattr(node, "answer", None)
            return f"LEAF\\nlabel={ans}\\\\nsamples={n_samples}"

        feat = getattr(node, "feature", None)
        thr = getattr(node, "threshold", None)
        gain = getattr(node, "gain", None)
        gini = getattr(node, "gini", None)
        lines = []
        if feat is not None:
            lines.append(f"feat: {feat}")
        if thr is not None:
            lines.append(f"thr: {thr}")
        if gain is not None and not (gain is None):
            # format numeric values shorter
            try:
                lines.append(f"gain:{round(float(gain), 4)}")
            except Exception:
                lines.append(f"gain:{gain}")
        if gini is not None and not (gini is None):
            try:
                lines.append(f"gini:{round(float(gini), 4)}")
            except Exception:
                lines.append(f"gini:{gini}")
        lines.append(f"samples={n_samples}")
        return "\\n".join(lines)

    def traverse(node, depth=0, parent_id=None, edge_label=None):
        nid = new_id()  # create label - use HTML-like label to keep newlines
        lbl = node_summary(node)
        dot.node(nid, label=f"<{safe_label(lbl)}>")

        if parent_id is not None:
            if edge_label is None:
                dot.edge(parent_id, nid)
            else:
                dot.edge(parent_id, nid, label=str(edge_label), fontsize="9")  # smaller font size for labels

        # stopping rules: max_display_depth or collapse_threshold
        if max_display_depth is not None and depth >= max_display_depth:
            # add a small collapsed marker node (no children)
            collapse_id = new_id()
            samples = 0
            ds = getattr(node, "datasample_labels", None)
            if ds is not None:
                try:
                    samples = len(ds)
                except Exception:
                    samples = 0
            dot.node(collapse_id, label=f"<{safe_label('... collapsed subtree\\nsamples=' + str(samples))}>", style="dashed")
            dot.edge(nid, collapse_id, style="dashed")
            return nid

        # If node has numeric threshold -> expected two children (left then right)
        thr = getattr(node, "threshold", None)
        children = getattr(node, "children", None) or []

        # if children list contains strings (your implementation sometimes uses children list of values for categorical),
        # but in _create_Tree you set children to Node objects. Here we assume they are Node objects.
        # handle if children are Node objects (usual case)
        for child in children:
            # safety: child might be None, or an object representing a terminal label; handle carefully
            if child is None:
                continue
            # Determine edge label (for numeric splits you set edge_value string, else categorical value)
            edge_val = getattr(child, "edge_value", None)
            traverse(child, depth+1, nid, edge_label=edge_val)
        return nid

    traverse(tree.root, depth=0, parent_id=None)

    # render to file (filename.pdf)
    outpath = dot.render(filename=filename, cleanup=True)  # cleanup=True removes intermediary .gv file
    print(f"Rendered tree to: {outpath}")
    return outpath

